# Monte Carlo Scanning
## Importing Libraries

In [1]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from tqdm import tqdm
from scipy.special import erf
import pickle
import itertools

from SALib.sample import saltelli
from SALib.analyze import sobol

# Personal libraries
import sixtrackwrap as sx

from parameters import *

## Computing

In [2]:
problem = {
    'num_vars': 3,
    'names': ['alpha', 'theta1', 'theta2'],
    'bounds': [[-1, 1],
               [0, np.pi * 2],
               [0, np.pi * 2]]
    }
param_values = saltelli.sample(problem, mc_max_samples // 8 + 8)

alpha = np.array([np.arccos(p[0])/2 for p in param_values])
theta1 = np.array([p[1] for p in param_values])
theta2 = np.array([p[2] for p in param_values])

DA_3 = {}
error_3 = {}
data_3 = {}
# Data generation
engine = sx.radial_scanner(alpha, theta1, theta2, d_r, starting_step=starting_step)
engine.scan(max_turns, min_turns)

radiuses = engine.extract_DA(turn_sampling)

data_3 = radiuses

# Computing DA

for sample in mc_samples:
    average = np.average(np.power(radiuses[:sample], 4), axis=0)
    error = np.std(np.power(radiuses[:sample], 4), axis=0) / np.sqrt(sample)
    DA_3[sample] = np.power(average, 1/4)
    error_3[sample] = 0.25 * np.power(average, -3/4) * error
    

Active radiuses: 1064 / 1064
Sample size per active radius: 2
Expected execution time for step: nan
Time per single iteration: 3.7111164483808936e-05
r: 11.0 . Turns to do: 1000 . Min found: 1000
Active radiuses: 1064 / 1064
Sample size per active radius: 2
Expected execution time for step: 78.97
Time per single iteration: 3.680831025865741e-05
r: 13.0 . Turns to do: 1000 . Min found: 1000
Active radiuses: 1064 / 1064
Sample size per active radius: 2
Expected execution time for step: 78.33
Time per single iteration: 3.708879884920622e-05
r: 15.0 . Turns to do: 1000 . Min found: 1000
Active radiuses: 1064 / 1064
Sample size per active radius: 2
Expected execution time for step: 78.92
Time per single iteration: 3.689084465342357e-05
r: 17.0 . Turns to do: 1000 . Min found: 59
Active radiuses: 1049 / 1064
Sample size per active radius: 2
Expected execution time for step: 77.40
Time per single iteration: 0.00020934025870833202
r: 19.0 . Turns to do: 1000 . Min found: 41
Active radiuses: 96

### Saving data

In [3]:
with open("data/DA_3.pkl", 'wb') as f:
    pickle.dump(DA_3, f, protocol=4)
    
with open("data/error_3.pkl", 'wb') as f:
    pickle.dump(error_3, f, protocol=4)